In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [16]:
data = pd.read_csv("https://github.com/YBI-Foundation/Dataset/raw/main/YouTube%20Spam.csv")
data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .,1


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1956 entries, 0 to 1955
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  1956 non-null   object
 1   AUTHOR      1956 non-null   object
 2   DATE        1711 non-null   object
 3   CONTENT     1954 non-null   object
 4   CLASS       1956 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 76.5+ KB


In [18]:
data.describe()

,CLASS
count,1956.000000
mean,0.513804
std,0.499937
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [19]:
data.columns

Index(['COMMENT_ID', 'AUTHOR', 'DATE', 'CONTENT', 'CLASS'], dtype='object')

#Train Test Split

In [20]:
x,tx = train_test_split(data, random_state=2529)
x = x[['CONTENT', 'CLASS']].values.tolist()

#Data Preprocessing

In [21]:
label = ['No', 'Yes']

nx = []
yx = []


for v in x:
  if isinstance(v[0], str):
    if v[1]==0:
      st = v[0].lower()
      nx = nx + re.findall(r'\b\w+\b', st)
    else:
      st = v[0].lower()
      yx.extend(re.findall(r'\b\w+\b', st))


frequency_n = {}
frequency_y = {}

for element in nx:
    if element in frequency_n:
        frequency_n[element] += 1
    else:
        frequency_n[element] = 1

for element in yx:
    if element in frequency_y:
        frequency_y[element] += 1
    else:
        frequency_y[element] = 1

#Naive Bayes Classifier

In [22]:
def isCommentSpam(st):
  input = re.findall(r'\b\w+\b', st)

  res = [1,1]

  for w in input:
    if w in frequency_y:
      p = frequency_y[w]
      q=p
      if w in frequency_n:
        q = q + frequency_n[w]
      res[1] = res[1]*p/q
    if w in frequency_n:
      p = frequency_n[w]
      q=p
      if w in frequency_y:
        q = q + frequency_y[w]
      res[0] = res[0]*p/q

  res[0] = res[0] * len(nx)/(len(nx)+len(yx))
  res[1] = res[1] * len(yx)/(len(nx)+len(yx))

  return label[np.argmax(res)]

In [23]:
val = isCommentSpam("A spouse surprises their partner with flowers and a romantic dinner.")
print(val)

Yes


#Testing the Model

In [24]:
tx = tx[['CONTENT', 'CLASS']].values.tolist()

conf = np.zeros((2,2))

for v in tx:
  if isinstance(v[0], str):
    if isCommentSpam(v[0]) == label[v[1]]:
      if label[v[1]]=='No':
        conf[0][0]+=1
      else:
        conf[1][1]+=1
    else:
      if label[v[1]]=='No':
        conf[1][0]+=1
      else:
        conf[0][1]+=1

print(f"Out of {conf[0][0]+conf[1][0]} labled as No {conf[0][0]} were correctly predicted")
print(f"Out of {conf[1][1]+conf[0][1]} labled as Yes {conf[1][1]} were correctly predicted")
print(conf[0][0]+conf[1][1])
print(conf[0][0]+conf[0][1]+conf[1][0]+conf[1][1])
print((conf[0][0]+conf[1][1])*100/(conf[0][0]+conf[0][1]+conf[1][0]+conf[1][1]))

Out of 230.0 labled as No 49.0 were correctly predicted
Out of 258.0 labled as Yes 258.0 were correctly predicted
307.0
488.0
62.90983606557377
